Sign Language Project Computer Vision Dataset

https://universe.roboflow.com/sign-language-colorful/sign-language-project-zxbft

In [2]:
# Install dependencies
%pip install -q ultralytics roboflow supervision pandas matplotlib opencv-python

You should consider upgrading via the '/Users/ahmadmiqdam/Desktop/capstone4/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Install library
import os
from dotenv import load_dotenv
from pathlib import Path
import yaml
import pandas as pd
import numpy as np
from roboflow import Roboflow
from ultralytics import YOLO
import shutil
import cv2

/Users/ahmadmiqdam/Desktop/capstone4/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Load API required
load_dotenv()
# Import Roboflow API
ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]

In [5]:
# Load first dataset (Object Detection)

rf_1 = Roboflow(api_key=ROBOFLOW_API_KEY)
project_1 = rf_1.workspace("test-hmtgo").project("sign-language-project-zxbft-ekfrd")
version_1 = project_1.version(1)
dataset_1 = version_1.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...


In [6]:
# Set out directory for object detection

HOME = os.getcwd()
base_path = Path(HOME)
first_dataset_location = base_path / "Sign-Language-Project-1"
yaml_data = first_dataset_location / "data.yaml"

In [7]:
# Function for restructuring data.yaml so it would convert from 36 class into 1 class (hand) only

def convert_yolo_labels_to_single_class(
    dataset_root: str,
    target_class_id: int = 0,
    dry_run: bool = False
):
    """
    Convert all YOLO label files in train/valid/test splits to a single class.
    """
    dataset_root = Path(dataset_root)
    splits = ["train", "valid", "test"]

    for split in splits:
        labels_dir = dataset_root / split / "labels"
        if not labels_dir.exists():
            continue

        print(f"\nProcessing {labels_dir}")

        for label_file in labels_dir.glob("*.txt"):
            with open(label_file, "r") as f:
                lines = f.readlines()

            new_lines = []
            changed = False

            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                if parts[0] != str(target_class_id):
                    parts[0] = str(target_class_id)
                    changed = True

                new_lines.append(" ".join(parts))

            if changed:
                if dry_run:
                    print(f"Dry Run Would update {label_file.name}")
                else:
                    with open(label_file, "w") as f:
                        f.write("\n".join(new_lines))
                    print(f"Updated {label_file.name}")


def update_data_yaml_to_single_class(
    yaml_path: str,
    class_name: str = "hand"
):
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    data["nc"] = 1
    data["names"] = {0: class_name}

    with open(yaml_path, "w") as f:
        yaml.safe_dump(data, f)

    print(f"Updated {yaml_path}")

In [9]:
# Convert all label files to class 0
convert_yolo_labels_to_single_class(
    dataset_root="Sign-Language-Project-1",
    target_class_id=0
)
# Convert data.yaml to 1-class
update_data_yaml_to_single_class(
    yaml_path="Sign-Language-Project-1/data.yaml",
    class_name="hand"
)


Processing Sign-Language-Project-1/train/labels

Processing Sign-Language-Project-1/valid/labels

Processing Sign-Language-Project-1/test/labels
Updated Sign-Language-Project-1/data.yaml


In [10]:
# Check GPU
!nvidia-smi

zsh:1: command not found: nvidia-smi


Object Detection

In [1]:
!wget https://universe.roboflow.com/test-hmtgo/handsign-5nz1l-ehtlc

--2026-01-15 15:46:37--  https://universe.roboflow.com/test-hmtgo/handsign-5nz1l-ehtlc
Resolving universe.roboflow.com (universe.roboflow.com)... 172.66.166.205, 104.20.41.123, 2606:4700:10::ac42:a6cd, ...
Connecting to universe.roboflow.com (universe.roboflow.com)|172.66.166.205|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2026-01-15 15:46:37 ERROR 403: Forbidden.



In [ ]:
# Model Comparison, pick the best model later

# 1. Base model
# (Pretrained YOLOv8s with no learning, no finetuning and augmentation as baseline)
model_1 = YOLO('yolov8s.pt')
results_1 = model_1.train(
    data=yaml_data,
    epochs=40,
    imgsz=640,
    batch=8,
    name='model_1',
    lr0=0.0,
    augment=False,
)

# 2. Fine-tuned
# Fine tune only, no augmentation
model_2 = YOLO('yolov8s.pt')
results_2 = model_2.train(
    data=yaml_data,
    epochs=40,
    imgsz=640,
    batch=8,
    name='model_2',
    lr0=0.001,
    augment=False,
)

# 3. Fine-tuned + no geometric augmentation
model_3 = YOLO('yolov8s.pt')
results_3 = model_3.train(
    data=yaml_data,
    epochs=40,
    imgsz=640,
    batch=8,
    name='model_3',
    lr0=0.001,
    # Zero geometric augmentations
    fliplr=0.0,
    flipud=0.0,
    mosaic=0.0,
    degrees=0.0,
    translate=0.0,
    scale=0.0,
    shear=0.0,
    # Allowed photometric augmentations
    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.4,
    erasing=0.1,
)

# 4. Fine-tuned + geometric augmentation
model_4 = YOLO('yolov8s.pt')
results_4 = model_4.train(
    data=yaml_data,
    epochs=40,
    imgsz=640,
    batch=8,
    name='model_4',
    lr0=0.001,
    fliplr=0.5,
    degrees=15,
    scale=0.5,
    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.4,
    erasing=0.1,
)

Ultralytics 8.4.2 🚀 Python-3.9.6 torch-2.8.0 CPU (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/ahmadmiqdam/Desktop/capstone4/Sign-Language-Project-1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=model_12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

In [ ]:
# Evaluation
all_metrics = []

models = [
    ("Model 1 – Base", model_1, "No fine-tune, no augmentation"),
    ("Model 2 – FT No Aug", model_2, "Fine-tune only"),
    ("Model 3 – FT Safe Aug", model_3, "Photometric augmentation"),
    ("Model 4 – FT Geo Aug", model_4, "Geometric augmentation"),
]

for name, model, desc in models:
    metrics = model.val()
    metrics_result = metrics.results_dict

    all_metrics.append({
        "Model": name,
        "Description": desc,
        "Precision": metrics_result["metrics/precision(B)"],
        "Recall": metrics_result["metrics/recall(B)"],
        "mAP50": metrics_result["metrics/mAP50(B)"],
        "mAP50-95": metrics_result["metrics/mAP50-95(B)"],
    })

metrics_df = pd.DataFrame(all_metrics)
display(metrics_df)

In [ ]:
# Download the best model (.pt)

def export_all_trained_models(
    runs_dir="runs/detect",
    model_names=("model_1", "model_2", "model_3", "model_4"),
    output_dir="exported_models",
    weight_name="best.pt"
):
    runs_dir = Path(runs_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    exported = []

    for model_name in model_names:
        src = runs_dir / model_name / "weights" / weight_name
        if not src.exists():
            print(f"[WARNING] {src} not found, skipping")
            continue

        dst = output_dir / f"{model_name}_{weight_name}"
        shutil.copy(src, dst)
        exported.append(dst.name)
        print(f"Exported: {dst}")

    return exported

In [ ]:
exported_models = export_all_trained_models()
print("Exported models:", exported_models)

In [ ]:
# So we will use the ... model because of blabla (download .pt nya)

Image Classification

In [ ]:
# Load second dataset (Image Classification)
rf_2 = Roboflow(api_key=ROBOFLOW_API_KEY)
project_2 = rf_2.workspace("test-hmtgo").project("handsign-5nz1l-ehtlc")
version_2 = project_2.version(1)
dataset_2 = version_2.download("folder")

In [22]:
# Set out directory for image classification

HOME = os.getcwd()
base_path = Path(HOME)
second_dataset_location = base_path / "HandSIgn-1"


In [23]:
ls {second_dataset_location}

README.dataset.txt  README.roboflow.txt  test/  train/  valid/


In [ ]:
# Load pretrained YOLOv8 classification backbone
model_cls = YOLO("yolov8n-cls.pt")

results = model_cls.train(
    data=second_dataset_location,   # dataset root
    epochs=40,
    imgsz=224,
    batch=32,
    name="handsign_cls_v1",
    lr0=0.001,
    optimizer="Adam",
)

In [ ]:
# Validation
metrics = model_cls.val()
print(metrics.results_dict)

Webcam integration

In [ ]:
model = YOLO("exported_models/handsign_cls_best.pt")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    res = model(frame, imgsz=224)[0]
    probs = res.probs

    label = model.names[probs.top1]
    conf = probs.top1conf.item()

    cv2.putText(
        frame,
        f"{label} ({conf:.2f})",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    cv2.imshow("Hand Sign Classification", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Streamlit / Gradio Interface